<a href="https://colab.research.google.com/github/gabrielcerono/DiabetesColaboration/blob/main/Feature_ranking_an_added_weight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
from sklearn.ensemble import RandomForestRegressor

In [2]:
data = pd.read_excel('/content/pone.0216416.s003.xlsx')

In [3]:
data = data.drop(['no'], axis=1)

In [4]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [5]:
iterative_imp = IterativeImputer()

In [6]:
iterative_imp.fit(data)

IterativeImputer(add_indicator=False, estimator=None,
                 imputation_order='ascending', initial_strategy='mean',
                 max_iter=10, max_value=None, min_value=None,
                 missing_values=nan, n_nearest_features=None, random_state=None,
                 sample_posterior=False, skip_complete=False, tol=0.001,
                 verbose=0)

In [7]:
data1 = iterative_imp.fit_transform(data)

In [8]:
data1 = pd.DataFrame(data=data1, columns= data.columns,)

In [9]:
data1.columns = ['sex', 'age', 'duration_of_diabetes', 'BMI',
       'insulin_regimen', 'TDD', 'TDD/kg', 'basal', 'basal/kg',
       'bolus', 'bolus/kg', 'HbA1c', 'eGFR', 'bodyfat', 'adiponectin',
       'free-test', 'SMI', 'grip_strength', 'knee_extension_strength',
       'gait_speed', 'ucOC', 'OC']

# ADDED_WEIGHT


In [10]:
data1['TDDkg'] = data1['TDD/kg']

In [11]:
data1['ADDED_WEIGHT'] = data1.TDD / data1.TDDkg

In [12]:
data1 = data1.drop(['basal/kg', 'bolus/kg', 'TDD/kg', 'TDDkg'], axis = 1)

# Feature Ranking


In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X = data1.drop('duration_of_diabetes', axis = 1)
y = data1['duration_of_diabetes']

In [15]:
X, X_test, y, y_test = train_test_split(X, y, test_size=0.30, random_state= 42)

In [28]:
def featureranking(X, y):
  X_train, X_fr, y_train, y_fr = train_test_split(X, y, test_size=0.30, random_state=30)
  rf = RandomForestRegressor(n_estimators=100, random_state=30)
  rf.fit(X_train, y_train)
  r2original = rf.score(X_fr, y_fr)
  
  r2fr= []
  columnsrf= []
  
  for x in X.columns:
    X_train, X_fr, y_train, y_fr = train_test_split(X, y, test_size=0.30, random_state=30)
    X_train = X_train.drop([x], axis=1)
    X_fr = X_fr.drop([x], axis=1)
    rf.fit(X_train, y_train)
    r2 = rf.score(X_fr, y_fr)
    columnsrf.append(x)
    r2fr.append(r2original - r2)
    
  resultado = np.array(list(zip(columnsrf, r2fr)))
  resultadopd = pd.DataFrame(data=resultado, columns=['Variables', 'r2-punish'])
  resultadopd.sort_values(by=['r2-punish'], ascending=False, inplace=True)
  return resultadopd

In [29]:
featureranking(X, y)

,Variables,r2-punish
1,age,0.42270496183206085
2,BMI,0.03866011450381668
15,gait_speed,0.026483587786259477
5,basal,0.014010687022900536
0,sex,0.010049999999999781
3,insulin_regimen,0.00841259541984718
4,TDD,0.0063620229007634865
12,SMI,0.00041717557251896853
18,ADDED_WEIGHT,0.0001337786259542062
9,bodyfat,-0.029409351145038176
